In [181]:
from flask import Flask, request
import json
import psycopg2
import psycopg2.extras
import os
import time
import requests
import pandas as pd
import json

In [189]:
api_host = os.getenv("API_HOST", "http://0.0.0.0:8080")
print(api_host)
respuesta = requests.get(api_host)
data_raw  = respuesta.json()
df        = pd.DataFrame.from_dict(pd.json_normalize(data_raw), orient="columns")
df

http://0.0.0.0:8080


,id,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species
0,0,5.1,3.5,1.4,0.2,Iris-setosa
1,1,4.9,3.0,1.4,0.2,Iris-setosa
2,2,4.7,3.2,1.3,0.2,Iris-setosa
3,3,4.6,3.1,1.5,0.2,Iris-setosa
4,4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
146,146,6.3,2.5,5.0,1.9,Iris-virginica
147,147,6.5,3.0,5.2,2.0,Iris-virginica
148,148,6.2,3.4,5.4,2.3,Iris-virginica
149,150,5.8,2.9,5.0,1.7,Iris-virginica


In [183]:
#get con filtro
id_filtro = 147
respuesta = requests.get(api_host+"/iris",{"id":id_filtro})
data_raw  = respuesta.json()
df        = pd.DataFrame.from_dict(pd.json_normalize(data_raw), orient="columns")
df

,id,sepallengthcm,sepalwidthcm,petallengthcm,petalwidthcm,species
0,147,6.5,3.0,5.2,2.0,Iris-virginica


In [184]:
#para hacer modificaciones
respuesta_post = requests.get(api_host)
data_raw_post  = respuesta_post.json()
df_post   = pd.DataFrame.from_dict(pd.json_normalize(data_raw_post), orient="columns")
#POST
#cada numero es un input
data_dic = {
"id": int(df_post["id"].max()+1), 
"sepallengthcm" : 5.8,
"sepalwidthcm"   : 2.9,
"petallengthcm"  : 5.0, 
"petalwidthcm"   : 1.7,
"species"        : "Iris-virginica"
}
data_dic = json.dumps(data_dic)
data_dic = f"[{data_dic}]"
requests.post(api_host+"/iris",data=data_dic)

<Response [200]>

In [178]:
#delete
id_filtro = 150
requests.delete(api_host+"/iris",params = {"id":id_filtro})


<Response [200]>

In [188]:
#patch
id_filtro = 149
data_dic = {
"sepallengthcm" : 5.8,
"sepalwidthcm"   : 2.9,
"petallengthcm"  : 5.0, 
"petalwidthcm"   : 1.7,
"species"        : "Iris-virginica"
}
data_dic = json.dumps(data_dic)
data_dic = f"[{data_dic}]"
requests.patch(api_host+"/iris",data = data_dic,params = {"id":id_filtro})

<Response [200]>